In [1]:
!pip install -r requirement.txt -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
!nvcc --version

'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!python -c "import torch; print(torch.cuda.is_available()); print(torch.version.cuda); print(torch.cuda.get_device_name(0))"

False
11.6


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "c:\Users\varun\anaconda3\envs\allennlp_env\lib\site-packages\torch\cuda\__init__.py", line 329, in get_device_name
    return get_device_properties(device).name
  File "c:\Users\varun\anaconda3\envs\allennlp_env\lib\site-packages\torch\cuda\__init__.py", line 359, in get_device_properties
    _lazy_init()  # will define _get_device_properties
  File "c:\Users\varun\anaconda3\envs\allennlp_env\lib\site-packages\torch\cuda\__init__.py", line 217, in _lazy_init
    torch._C._cuda_init()
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx


In [4]:
!python -c "from allennlp.modules.feedforward import FeedForward; print('✅ All good')"


✅ All good


In [5]:
!python tokenizer.py

In [6]:
!python layers.py

In [7]:
!python model.py

In [2]:
from rdkit import Chem
from model import MolGen

# load data
data = []
with open('qm9.csv', "r") as f:
    for line in f.readlines()[1:]:
        data.append(line.split(",")[1])

# create model
gan_mol = MolGen(data, hidden_dim=64, lr=1e-3, device="cpu")

c:\Users\varun\anaconda3\envs\allennlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# create dataloader
loader = gan_mol.create_dataloader(data, batch_size=128, shuffle=True, num_workers=10)

# train model for 20000 steps
gan_mol.train_n_steps(loader, max_step=20000, evaluate_every=100)

# save the model after training
import torch
import os

# Create a directory for saving models if it doesn't exist
os.makedirs('saved_models', exist_ok=True)

# Save generator and discriminator if available
torch.save(gan_mol.generator.state_dict(), 'saved_models/generator.pth')
torch.save(gan_mol.discriminator.state_dict(), 'saved_models/discriminator.pth')

print("Model saved successfully.")

In [ ]:
gan_mol.eval()

print('ok')

In [ ]:
# After training
# generate Smiles molecules
smiles_list = gan_mol.generate_n(8)

# convert with rdkit
mol_list = [Chem.MolFromSmiles(m) for m in smiles_list]

# draw
Chem.Draw.MolsToGridImage(mol_list, molsPerRow=4, subImgSize=(250, 250), maxMols=10)